# In-Class Assignment: Bacterial Colony Counting

<img src="https://lh6.googleusercontent.com/Y0ztWMEmnMlvCb6eQCasP7ZzqO8G8nzh_0aeTtPJoXBudt5b6f5kdA0io9Knfiva1IoCQaxOog=w580" width="65%">


### Agenda for today's class (80 minutes)


1. [Review pre-class assignment](#class_Assignment)
2. [Download Petri Dish images for notebook](#Download_Petri_Dish_images_for_notebook)
3. [Establishing "Ground Truth"](#Establishing_Ground_Truth)
2. [Automated Colony Counting](#Automated_Colony_counting)


---
<a name="Download_Petri_Dish_images_for_notebook"></a>

# 1. Download Petri Dish images for notebook

In [ ]:
#The following code snip-it downloads a file from internet and saves it to your local directory.
%matplotlib inline
import numpy as np
from scipy import misc, ndimage
from imageio import imread
import matplotlib.pylab as plt
from urllib.request import urlopen, urlretrieve

url = 'https://goo.gl/cebwc1'
file1 = 'Petri_dish1.jpg'
urlretrieve(url, file1);

url = 'https://goo.gl/P82yKZ'
file2 = 'Petri_dish2.jpg'
urlretrieve(url, file2);

im1 = imread(file1)
im2 = imread(file2)

f, (ax1, ax2) = plt.subplots(1, 2,figsize=(20,10))
ax1.imshow(im1)
ax1.axis('off')
ax1.set_title('Petri_dish1')
ax2.imshow(im2)
ax2.axis('off')
ax2.set_title('Petri_dish2');

&#9989; <font color=red>**DO THIS:**</font> Modify the following code to select petri dish 2 instead of petri dish 1

In [ ]:
#Select your petri Dish (either im1 or im2)
#Note this is just making a new name for the same variable...
# i.e. any changes made to im are also made to im1
# to make two different objects you need to do a copy (see below)

im = im1

----
<a name="Establishing_Ground_Truth"></a>

# 2. Establishing "Ground Truth"

For this assignment we will only count the total colonies your selected dish.  Consider the image below.  How do we determine if we have the "right" answer?

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(im)
plt.axis('off')

&#9989; <font color=red>**DO THIS:**</font> - record your manual findings for colony counts here:

In [ ]:
# Store the estimates of colonie counts as python vectors (comma seperated values). 
# Note the values provided below are just random numbers selected to make the program do something.
# Your group should change these numbers


Manual_Colony_Count = [ 387, 402, 339 ]

----
<a name="Automated_Colony_counting"></a>
# 3. Automated Colony counting
The following code attempts to automatically count the colonies in petri dish 1. The code has four basic parts:

1. Background Subtraction - Remove everything that is not the petri dish.
2. Colony Selection - Separate the colonies from the dish.
3. Count Blobs - Use the labeling algorithm to count the colonies.
4. Graph Results - Compare Manual Selection with Automated selection.

You will modify the code as you see fit to clean up the segmentation and get the most accurate count.  


### Step 1. Background Subtraction
Notice in the first image (im1) the background near the top of the image outside of the pitri dish has a color similar to the colonies of interest.  The first thing we are going to do is create a mask identify everyting outisde of the pitri dish so we can easily ignore it later.  Notice that a simple color threshold of 133 results in true mostly outside of the dish and false inside of the dish:

### Simple color threshold

In [ ]:
rmax=133
background_mask = im[:,:,0] < rmax 

# Remove the background_mask from the original image
forground_im = im.copy()
forground_im[background_mask,:] = 0

#Plot the results side-by-side with the original image
f, (ax1, ax2) = plt.subplots(1, 2,figsize=(20,10))
ax1.imshow(background_mask, cmap='gray')
ax1.set_title('Background Mask')
ax1.axis('off')
ax2.imshow(forground_im)
ax1.set_title('Masked Image')

### Crop mask

We still have some large errors at the top.  We could just chop off this part of the image by setting all of the image rows less than 90 to true:

In [ ]:
# Trick to trim off top of dish
background_mask[0:90,:] = 1

# Remove the background_mask from the original image
forground_im = im.copy()
forground_im[background_mask,:] = 0

#Plot the results side-by-side with the original image
f, (ax1, ax2) = plt.subplots(1, 2,figsize=(20,10))
ax1.imshow(background_mask, cmap='gray')
ax1.set_title('Background Mask')
ax1.axis('off')
ax2.imshow(forground_im)
ax1.set_title('Masked Image')

### Clean up the noise

Now we can use dilation to get rid of the small black dots in the mask.  

In [ ]:
from scipy import ndimage

after_dilation = ndimage.binary_dilation(background_mask, iterations=10)


# Remove the background_mask from the original image
forground_im = im.copy()
forground_im[after_dilation,:] = 0

#Plot the results side-by-side with the original image
f, (ax1, ax2) = plt.subplots(1, 2,figsize=(20,10))
ax1.imshow(after_dilation, cmap='gray')
ax1.set_title('Background Mask')
ax1.axis('off')
ax2.imshow(forground_im)
ax1.set_title('Masked Image')

----

### Step 2. Colony Selection

Using the background mask from above we can now focus only on the petri dish.  This code tries to segment out both the TypeA and TypeB colonies.

In [ ]:
im = forground_im

In [ ]:
#Code snipit from previous class to show image as HSV
import colorsys
import matplotlib.colors as colors

hsv = colors.rgb_to_hsv(im)
f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4,figsize=(20,5))
ax1.imshow(hsv[:,:,0], cmap='hsv')
ax1.set_title('Hue - Color')
ax1.set_axis_off()

ax2.imshow(hsv[:,:,1],cmap='gray',vmin=0, vmax=1)
ax2.set_title('Saturation - Amount of Color')
ax2.set_axis_off()

ax3.imshow(hsv[:,:,2],cmap='gray')
ax3.set_title('Value - Brightness')
ax3.set_axis_off()

ax4.imshow(im)
ax4.set_axis_off()

In [ ]:
#Code Snipit from previous class to show H channel of HSV with a colorbar
plt.figure(figsize=(10,5));
plt.imshow(hsv[:,:,0], cmap='hsv');
plt.colorbar();

In [ ]:
#Code snipit from previous class to find best thresholds for HSV Colorspace
from ipywidgets import interactive,fixed

def hsv_color_threshold(im, hmin=-0.01,hmax=1.01, smin=-0.01,smax=1.01,vmin=-1,vmax=256):
    # Pull out the red, gree and blue matrixes
    hsv = colors.rgb_to_hsv(im)
    h = hsv[:,:,0];
    s = hsv[:,:,1];
    v = hsv[:,:,2];
    
    # trick because the color space wraps
    if hmin > hmax:
        b_img = (h > hmin) | (h < hmax)
    else:
        b_img = (h > hmin) & (h < hmax);
    
    
    b_img = (b_img & 
         (s > smin) & (s < smax) & 
         (v > vmin) & (v < vmax));
    
    f, (ax1, ax2) = plt.subplots(1, 2,figsize=(20,5))
    ax1.imshow(im)
    ax1.set_axis_off()
    
    ax2.imshow(b_img,cmap='gray', vmin=0, vmax=1)
    ax2.set_title('Value - Brightness')
    ax2.set_axis_off()

    plt.show()
    return b_img*1;

slider_widget = interactive(hsv_color_threshold, im=fixed(im),
                            hmin=(-0.01,1.01,0.01), hmax=(-0.01,1.01,0.01), 
                            smin=(-0.01,1.01,0.01), smax=(-0.01,1.01,0.01), 
                            vmin=(-1,256), vmax=(-1,256),__manual=True);
slider_widget

In [ ]:
#THis code pulls out the thresholds from the slider widget used above
hmin = slider_widget.children[0].value
hmax = slider_widget.children[1].value
smin = slider_widget.children[2].value
smax = slider_widget.children[3].value
vmin = slider_widget.children[4].value
vmax = slider_widget.children[5].value

print('hmin=',hmin)
print('hmax=',hmax)
print('smin=',smin)
print('smax=',smax)
print('vmin=',vmin)
print('vmax=',vmax)


If you stop and restart this notebook the threshold values will get reset. Copy and paste the best values from above into the following cell so you will not loose them. 

In [ ]:
# Type your thresholds here so you can remember what you picked (should be able to just copy and paste from above)
hmin= 0.86
hmax= 1.01
smin= 0.05
smax= 1.01
vmin= 137
vmax= 256

In [ ]:
# Code to visualize the segmentation from above
hsv = colors.rgb_to_hsv(im)

h1 = hsv[:,:,0] > hmin 
h2 = hsv[:,:,0] < hmax 
s1 = hsv[:,:,1] > smin
s2 = hsv[:,:,1] < smax
v1 = hsv[:,:,2] > vmin
v2 = hsv[:,:,2] < vmax

binary_image = h1 & h2 & s1 & s2 & v1 & v2 & ~(after_dilation)


im_background = im.copy()
im_background[binary_image] = 0

#Plot the results side-by-side with the original image
f, (ax1, ax2) = plt.subplots(1, 2,figsize=(20,10))
ax1.imshow(im_background)
ax1.axis('off')
ax2.imshow(binary_image, cmap='gray')
ax2.axis('off')


### Clean up the noise

Remove speckle noise using dilation and Erosion

In [ ]:

after_dilation = ndimage.binary_dilation(binary_image, iterations=2)
after_erosion  = ndimage.binary_erosion(after_dilation, iterations=2)


# Remove the background_mask from the original image
forground_im = im.copy()
forground_im[after_dilation,:] = 0

#Plot the results side-by-side with the original image
f, (ax1, ax2) = plt.subplots(1, 2,figsize=(20,10))

ax1.imshow(forground_im)
ax1.set_title('Background Image')

ax2.imshow(~after_dilation, cmap='gray')
ax2.set_title('Background Mask')
ax2.axis('off')

### Check results

Which colonies did we miss?

In [ ]:
im_background = im.copy()
im_background[after_dilation] = 0

#Plot the results side-by-side with the original image
f, (ax1, ax2) = plt.subplots(1, 2,figsize=(20,10))
ax1.imshow(im_background, cmap='gray')
ax1.axis('off')
ax2.imshow(after_dilation)
ax2.axis('off')

----
### Step 3. Object Counting

We can count the number of objects in the image using the label function inside of ```ndimage.measurements``` library:

In [ ]:
lab, num_features = ndimage.measurements.label(binary_image)
plt.imshow(lab, cmap='jet')
plt.colorbar()
plt.title(('A Total of',num_features,'colonies found'))

----

### Step 4. Graph Results

This final step will take the results from above and graph them as a bar chart for comparison.

In [ ]:
#Calculate the mean and standard deviation for manual colony counts
N = len(Manual_Colony_Count)
m_mean = np.mean(Manual_Colony_Count)
m_std = np.std(Manual_Colony_Count)

In [ ]:
# Plot the results

ind = np.arange(2)  # the x locations for the groups

fig, ax = plt.subplots(figsize=(10,5))
rects1 = ax.bar(0, m_mean, color='green', yerr=m_std)
rects2 = ax.bar(1, num_features, color='red')

# add some text for labels, title and axes ticks
ax.set_ylabel('Number of Colonies')
ax.set_title('Counting Comparison')
ax.set_xticks([0,1])
ax.set_xticklabels(('Manual', 'Automated'))

----
Written by Dr. Dirk Colbry, Michigan State University
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.

----